# DOGS ADOPTIONS PROJECT

**Importing libraries**

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', 50)
pd.options.display.max_columns = 40
pd.options.display.max_rows = 700

**Importing the 'dogs', 'dog_travel' and 'NST_EST' dataframe:**

In [3]:
dogs = pd.read_csv("adoptions/dogs.csv")

In [4]:
dog_travel = pd.read_csv("adoptions/dogTravel.csv")

In [5]:
NST_EST = pd.read_csv("adoptions/NST-EST2021-POP.csv", header=None)
NST_EST.columns = ['state', 'population']

**Converts the values in 'population column' from strings to integers**

In [6]:
NST_EST['population'] = NST_EST['population'].str.replace('.', '', regex=False).astype(int)

#The str.replace() method replaces any dots in the population column with an empty string,
#and then the astype() method converts the resulting strings to integers

**Importing the 'states' dataset, which contains all the USA states and their abbreviation (this dataset will be used in exercise 5 and 9)**

In [7]:
states = pd.read_csv("adoptions/states.csv", sep=';')

states['zip_end'] = states['zip_end'].fillna(0)
states['zip_end'] = states['zip_end'].astype(int)

states['zip_start'] = states['zip_start'].fillna(0)
states['zip_start'] = states['zip_start'].astype(int)

states.head()

,state,abbreviation,zip_start,zip_end
0,Alabama,AL,35004,36925
1,Alaska,AK,99501,9950
2,Arizona,AZ,85001,86556
3,Arkansas,AR,71601,72959
4,California,CA,90001,96162


# PRE-PROCESSING

**Cleaning the 'name' column in the 'dogs' dataframe**

**In the following rows, column 'status' contains the dog name in a "dirty" form and also what should be contained in the 'description' column**

In [8]:
names_and_desc = dogs[dogs['status'] != 'adoptable']['name'] 
names_and_desc.head()

644      Gunther :Gunny\",Meet handsome 3 year old Gunt...
5549     ANNABELLE \ANNIE\","You can fill out an adopti...
10888    PEPPER \Courtesy listing\","This is Pepper. He...
11983    COOPER \courtesy listing\","Cooper is 13 years...
12495    DAISY \courtesy listing\","•Basset Hound, fema...
Name: name, dtype: object

**Isolate the names in "dirty" form in a separate list:**

In [9]:
dirty_names = []
for name_desc in names_and_desc:   #iterate through each element in 'names_and_desc'
    dirty_names.append(name_desc.split(sep=",")[0])   #split name and desctiption and append only the
dirty_names

['Gunther :Gunny\\"',
 'ANNABELLE \\ANNIE\\"',
 'PEPPER \\Courtesy listing\\"',
 'COOPER \\courtesy listing\\"',
 'DAISY \\courtesy listing\\"',
 'Elmo \\MoMo\\"',
 'Bianca \\Pinky\\"',
 '\\Baby Girl\\"',
 'King Bert \\Bertie\\"',
 'Maddie \\Cutie Patootie!\\"',
 'Bucky \\Are u my hooman?\\"',
 'CHLOE \\LITTLE RASCAL\\"',
 'COZI H WILSON \\loves children',
 'Gibson Edgar \\Gibbs\\"',
 'PUPPIES \\Berries\\"',
 'Leila \\COURTESY POST\\"',
 'Eden \\In Training\\"',
 'Coach Chris \\Mack\\"',
 'Liberty \\Libby\\"',
 'Samson \\Sam\\"',
 'Sully \\Sullivan\\"',
 'PERIWINKLE \\WINK\\"',
 'Raymond \\Reddington\\"',
 'Norma Rose \\ROSIE\\"',
 'Markey \\Moose\\"',
 'George \\Boy George\\"',
 '\\Skipper\\"',
 '\\Bella\\"',
 '\\Cody\\"',
 '\\Gracie\\"',
 '\\Jameson\\"',
 '\\Canelo\\"',
 '\\Noni\\"']

There are two pattern in the diry names: 
1. name \\\nickname-char\\\\"
2. \\\nickname-char\\\\"

in the first case we want to isolate the name

in the second case we isolate the nickname

In [10]:
nomi = []
for name in dirty_names:
    match = re.split(r'[:\\]', name)    #matches either a colon (:) or a backslash (\) and split 'name' where it is matched
                                        #stores the results in a list called 'match'            
    if match[0] == '':                  #pattern n° 2 -> the 'match' list contains only the nickname
        nomi.append(match[1].strip())   #remove leading or trailing spaces with 'strip()' and appending to the 'nomi' list
    else:                               #pattern n° 1 -> the 'match' list contains only the nickname
        nomi.append(match[0].strip()) 
dogs_clean = dogs.copy(deep=True)       #create a copy of the 'dogs' dataframe ('dogs_clean') that is completely 
                                        #indipendent from it (thanks to deep=True): if you modify 'dogs' this will
                                        # not affect 'dogs_clean'

dogs_clean.loc[dogs_clean['status'] != 'adoptable', 'name'] = nomi 
# ^ assign the cleaned names in the column 'name' of dogs_clean

dogs_clean[dogs_clean['status'] != 'adoptable']['name']

644            Gunther
5549         ANNABELLE
10888           PEPPER
11983           COOPER
12495            DAISY
12600             Elmo
12613           Bianca
17619        Baby Girl
18611        King Bert
19747           Maddie
19845            Bucky
22161            CHLOE
22229    COZI H WILSON
29283     Gibson Edgar
30471          PUPPIES
31581            Leila
31888             Eden
33000      Coach Chris
33527          Liberty
34188           Samson
35065            Sully
44830       PERIWINKLE
53168          Raymond
53539       Norma Rose
55434           Markey
55467           George
55915          Skipper
55975            Bella
56013             Cody
56248           Gracie
56464          Jameson
56473           Canelo
56541             Noni
Name: name, dtype: object

**Shift the columns corresponding to the previous names from 'status' to 'accessed' of one to the right**

In [11]:
dogs_clean.loc[dogs_clean['status'] != 'adoptable','status':'accessed'] = dogs_clean.loc[dogs_clean['status'] != 'adoptable','status':'accessed'].shift(1, axis=1)
dogs_clean.loc[dogs_clean['status'] != 'adoptable']

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Young,Male,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,Gunther,None,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,White / Cream,NaN,Adult,Female,Large,Short,True,True,NaN,False,True,NaN,NaN,False,ANNABELLE,None,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,True,True,True,PEPPER,None,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,NaN,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,False,COOPER,None,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,White / Cream,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True,False,False,NaN,DAISY,None,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,NaN,NaN,Senior,Male,Large,Short,True,True,NaN,False,True,True,True,NaN,Elmo,None,2019-07-11T20:34:42+0000,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,NaN,NaN,Senior,Female,Medium,Short,True,True,NaN,False,True,True,True,NaN,Bianca,None,2019-07-11T14:16:38+0000,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,NaN,NaN,Senior,Female,Small,Short,True,True,NaN,False,True,True,True,NaN,Baby Girl,None,2019-08-10T16:00:35+0000,Bristow,VA,20136,US,DC,2019-09-20,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,NaN,NaN,Young,Male,Small,Short,True,False,NaN,False,True,False,NaN,False,King Bert,None,2019-05-14T21:09:27+0000,Silver Spring,MD,20905,US,DC,2019-09-20,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,NaN,NaN,Adult,Female,Large,NaN,True,True,NaN,False,True,NaN,NaN,False,Maddie,None,2016-12-15T13:33:43+0000,Gettysburg,PA,17325,US,DC,2019-09-20,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


In [12]:
dogs_clean.loc[17610:17630,'status':'accessed']

,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed
17610,adoptable,2019-08-10T22:33:34+0000,Chambersburg,PA,17201,US,DC,2019-09-20
17611,adoptable,2019-08-10T20:56:53+0000,Pennsville,NJ,08070,US,DC,2019-09-20
17612,adoptable,2019-08-10T19:50:26+0000,Hagerstown,MD,21740,US,DC,2019-09-20
17613,adoptable,2019-08-10T19:49:23+0000,Springfield,VA,22153,US,DC,2019-09-20
17614,adoptable,2019-08-10T19:31:25+0000,Hagerstown,MD,21740,US,DC,2019-09-20
17615,adoptable,2019-08-10T17:47:16+0000,Paw Paw,WV,25434,US,DC,2019-09-20
17616,adoptable,2019-08-10T16:46:30+0000,Bristow,VA,20136,US,DC,2019-09-20
17617,adoptable,2019-08-10T16:21:20+0000,Baltimore,MD,21224,US,DC,2019-09-20
17618,adoptable,2019-08-10T16:08:13+0000,Baltimore,MD,21224,US,DC,2019-09-20
17619,None,2019-08-10T16:00:35+0000,Bristow,VA,20136,US,DC,2019-09-20


# 1. Extract all dogs with status that is not adoptable

**Dataset before cleaning:**

In [13]:
dogs[dogs['status'] != 'adoptable'][['id', 'name', 'status']].head()

,id,name,status
644,41330726,"Gunther :Gunny\"",Meet handsome 3 year old Gunt...",2018-04-05T05:18:31+0000
5549,38169117,"ANNABELLE \ANNIE\"",""You can fill out an adopti...",2017-05-26T21:43:16+0000
10888,45833989,"PEPPER \Courtesy listing\"",""This is Pepper. He...",2019-09-01T15:12:06+0000
11983,45515547,"COOPER \courtesy listing\"",""Cooper is 13 years...",2019-08-06T12:15:58+0000
12495,45294115,"DAISY \courtesy listing\"",""•Basset Hound, fema...",2019-07-18T14:20:58+0000


**Cleaned dataset:**

In [14]:
not_ad_dogs = dogs_clean[dogs_clean['status'] != 'adoptable'][['id', 'name', 'status']]
not_ad_dogs.head()

,id,name,status
644,41330726,Gunther,None
5549,38169117,ANNABELLE,None
10888,45833989,PEPPER,None
11983,45515547,COOPER,None
12495,45294115,DAISY,None


# 2. For each (primary) breed, determine the number of dogs

In [15]:
breed_counts = dogs.groupby('breed_primary').count()[['id']].rename(columns={'id':'counts'})#.reset_index()[['breed_primary','counts']]
breed_counts

,counts
breed_primary,
Affenpinscher,17
Afghan Hound,4
Airedale Terrier,19
Akbash,3
Akita,181
Alaskan Malamute,72
American Bulldog,1134
American Eskimo Dog,43
American Foxhound,17


# 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed

In [16]:
dogs_clean.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,Yellow / Tan / Blond / Fawn,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,NaN,HARLEY,adoptable,2019-09-20T16:37:59+0000,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,White / Cream,NaN,Adult,Male,Large,Short,True,True,NaN,False,True,NaN,NaN,NaN,BIGGIE,adoptable,2019-09-20T16:24:57+0000,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,NaN,NaN,Adult,Male,Large,Short,True,False,NaN,False,True,NaN,NaN,NaN,Ziggy,adoptable,2019-09-20T14:10:11+0000,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,NaN,NaN,Baby,Female,Large,NaN,False,False,NaN,False,False,NaN,NaN,NaN,Gypsy,adoptable,2019-09-20T10:08:22+0000,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,NaN,NaN,Young,Male,Small,Long,True,False,NaN,False,True,True,True,True,Theo,adoptable,2019-09-20T06:48:30+0000,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


**METHOD 1:**

**Considering the 'breed_secondary' column: if it contains the text 'Mixed Breed' it means that the dog is of mixed breed, otherwise it is not**

**Selecting the 'Mixed Breed' dogs**:

In [17]:
mixed_for_breed = dogs_clean[dogs_clean['breed_secondary'] == 'Mixed Breed'].groupby('breed_primary', as_index=False).count()
mixed_for_breed['n_mixed'] = mixed_for_breed['id']
mixed_for_breed = mixed_for_breed[['breed_primary', 'n_mixed']]
mixed_for_breed.head()

,breed_primary,n_mixed
0,Affenpinscher,1
1,Airedale Terrier,1
2,Akita,6
3,Alaskan Malamute,4
4,American Bulldog,106


**Selecting the 'Not of Mixed Breed' dogs**:

In [18]:
not_mixed_for_breed = dogs_clean[dogs_clean['breed_secondary'] != 'Mixed Breed'].groupby('breed_primary', as_index=False).count()
not_mixed_for_breed['n_not_mixed'] = not_mixed_for_breed['id']
not_mixed_for_breed = not_mixed_for_breed[['breed_primary', 'n_not_mixed']]
not_mixed_for_breed

,breed_primary,n_not_mixed
0,Affenpinscher,16
1,Afghan Hound,4
2,Airedale Terrier,18
3,Akbash,3
4,Akita,175
5,Alaskan Malamute,68
6,American Bulldog,1028
7,American Eskimo Dog,41
8,American Foxhound,17
9,American Hairless Terrier,2


**Merging the tables**:

In [19]:
ratio_mixed = pd.merge(mixed_for_breed, not_mixed_for_breed, on='breed_primary')
ratio_mixed

,breed_primary,n_mixed,n_not_mixed
0,Affenpinscher,1,16
1,Airedale Terrier,1,18
2,Akita,6,175
3,Alaskan Malamute,4,68
4,American Bulldog,106,1028
5,American Eskimo Dog,2,41
6,American Hairless Terrier,2,2
7,American Staffordshire Terrier,110,1752
8,Anatolian Shepherd,9,109
9,Australian Cattle Dog / Blue Heeler,72,901


**Calculating the ratio between 'mixed' and 'not_mixed'**:

In [20]:
ratio_mixed['ratio_M_notM'] = ratio_mixed['n_mixed']/not_mixed_for_breed['n_not_mixed']
ratio_mixed

,breed_primary,n_mixed,n_not_mixed,ratio_M_notM
0,Affenpinscher,1,16,0.062500
1,Airedale Terrier,1,18,0.250000
2,Akita,6,175,0.333333
3,Alaskan Malamute,4,68,1.333333
4,American Bulldog,106,1028,0.605714
5,American Eskimo Dog,2,41,0.029412
6,American Hairless Terrier,2,2,0.001946
7,American Staffordshire Terrier,110,1752,2.682927
8,Anatolian Shepherd,9,109,0.529412
9,Australian Cattle Dog / Blue Heeler,72,901,36.000000


**METHOD 2:**

**Considering the 'breed_mixed' column**

**If True it means that the dog is of mixed breed**

In [21]:
mixed_breed = dogs_clean.where(dogs_clean['breed_mixed']== True).groupby('breed_primary').size().reset_index(name='mixed')
mixed_breed.head()

,breed_primary,mixed
0,Affenpinscher,5
1,Afghan Hound,4
2,Airedale Terrier,17
3,Akbash,2
4,Akita,83


**If False it means that the dog is not of mixed breed**

In [22]:
not_mixed_breed = dogs.where(dogs['breed_mixed'] == False).groupby('breed_primary').size().reset_index(name='not_mixed')
not_mixed_breed.head()

,breed_primary,not_mixed
0,Affenpinscher,12
1,Airedale Terrier,2
2,Akbash,1
3,Akita,98
4,Alaskan Malamute,52


**We merge and calculate the ratio**

In [23]:
breed_ratio = pd.merge(mixed_breed, not_mixed_breed, on='breed_primary')
breed_ratio.head()

,breed_primary,mixed,not_mixed
0,Affenpinscher,5,12
1,Airedale Terrier,17,2
2,Akbash,2,1
3,Akita,83,98
4,Alaskan Malamute,20,52


In [24]:
breed_ratio['ratio'] = breed_ratio['mixed'] / breed_ratio['not_mixed']
breed_ratio.head()

,breed_primary,mixed,not_mixed,ratio
0,Affenpinscher,5,12,0.416667
1,Airedale Terrier,17,2,8.500000
2,Akbash,2,1,2.000000
3,Akita,83,98,0.846939
4,Alaskan Malamute,20,52,0.384615



# 4. For each (primary) breed, determine the earliest and the latest posted timestamp.


**We check and convert the 'posted' type to datetime**

In [25]:
print(dogs_clean['posted'].dtypes)

object


In [26]:
dogs_clean['posted'] = pd.to_datetime(dogs_clean['posted'])

In [27]:
dogs_clean.dropna(subset=['posted'], inplace=True)
len(dogs_clean)

58180

In [28]:
print(dogs_clean['posted'].dtypes)

datetime64[ns, UTC]


**We find the min and the max for each 'breed_primary' and merge them in a single table**

In [29]:
earliest_timestamps = dogs_clean.groupby(by='breed_primary')['posted'].min().reset_index()
latest_timestamps = dogs_clean.groupby(by='breed_primary')['posted'].max().reset_index()

In [30]:
timestamps = pd.merge(earliest_timestamps, latest_timestamps, on="breed_primary").rename(columns={'posted_x':'earliest_timestamp', 'posted_y':'latest_timestamp'})
timestamps.head()

,breed_primary,earliest_timestamp,latest_timestamp
0,Affenpinscher,2012-03-08 10:27:33+00:00,2019-09-14 10:10:51+00:00
1,Afghan Hound,2017-06-29 23:28:51+00:00,2019-07-27 00:38:48+00:00
2,Airedale Terrier,2014-06-13 12:59:36+00:00,2019-09-19 18:40:39+00:00
3,Akbash,2019-07-21 00:35:59+00:00,2019-08-23 17:11:04+00:00
4,Akita,2012-03-03 09:31:08+00:00,2019-09-20 15:19:57+00:00


# 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

**We count the values of Females and Males for each State and then calculate the sex imbalance**

In [31]:
sex_counts = dogs_clean.groupby('contact_state')['sex'].value_counts().unstack()
sex_counts.head()

sex,Female,Male,Unknown
contact_state,,,
AK,7.0,8.0,NaN
AL,716.0,712.0,NaN
AR,351.0,344.0,NaN
AZ,1067.0,1181.0,1.0
CA,777.0,887.0,NaN


In [32]:
sex_counts['sex_imbalance'] = sex_counts['Male'] - sex_counts['Female']
sex_counts.head()

sex,Female,Male,Unknown,sex_imbalance
contact_state,,,,
AK,7.0,8.0,NaN,1.0
AL,716.0,712.0,NaN,-4.0
AR,351.0,344.0,NaN,-7.0
AZ,1067.0,1181.0,1.0,114.0
CA,777.0,887.0,NaN,110.0


**Using the 'states' df, we find the full name of the state with the largest sex imbalance, through the 'abbreviation' column**

In [33]:
max_imbalance_state = sex_counts['sex_imbalance'].idxmax()
max_imbalance_state_name = states.loc[states['abbreviation'] == max_imbalance_state].values[0][0]
print(f'The state with the largest sex imbalance is: {max_imbalance_state_name}')

The state with the largest sex imbalance is: Ohio


# 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [34]:
mean_stay_cost = dogs_clean.groupby(['age', 'size'], as_index=False)[['age','size','stay_duration','stay_cost']].mean(['stay_duration','stay_cost'])
mean_stay_cost

,age,size,stay_duration,stay_cost
0,Adult,Extra Large,89.015414,232.591561
1,Adult,Large,89.531943,238.661141
2,Adult,Medium,89.421036,238.258977
3,Adult,Small,89.407479,238.974838
4,Baby,Extra Large,87.032967,237.180879
5,Baby,Large,89.701564,238.698827
6,Baby,Medium,89.577668,237.108131
7,Baby,Small,89.958291,239.083810
8,Senior,Extra Large,88.861111,235.232361
9,Senior,Large,88.984298,237.507364


**Apply a categorization to the values contained in the 'age' column:**

In [35]:
mean_stay_cost['age'] = pd.Categorical(mean_stay_cost['age'], 
                                       categories=['Baby', 'Young', 'Adult', 'Senior'],
                                       ordered=True
                                      )

mean_stay_cost['size'] = pd.Categorical(mean_stay_cost['size'], 
                                        categories = ['Small', 'Medium', 'Large', 'Extra Large'],
                                        ordered=True
                                       )

**Renaming the columns:**

In [36]:
mean_stay_cost.rename(columns={"stay_duration": "mean_stay_duration", "stay_cost": "mean_stay_cost"}, inplace=True)

In [37]:
mean_stay_cost.groupby(by=['age', 'size'])[['mean_stay_duration', 'mean_stay_cost']].mean() 
#use 'age' and 'size' as index and sorting to show the results

mean_stay_duration  mean_stay_cost
age    size                                           
Baby   Small                 89.958291      239.083810
       Medium                89.577668      237.108131
       Large                 89.701564      238.698827
       Extra Large           87.032967      237.180879
Young  Small                 89.814275      241.540069
       Medium                89.515123      239.304603
       Large                 90.104206      238.149506
       Extra Large           90.586345      245.835582
Adult  Small                 89.407479      238.974838
       Medium                89.421036      238.258977
       Large                 89.531943      238.661141
       Extra Large           89.015414      232.591561
Senior Small                 89.073626      238.282286
       Medium                89.810052      238.514615
       Large                 88.984298      237.507364
       Extra Large           88.861111      235.232361

# 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [38]:
dog_travel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


**We count the number of travels for each 'id' and select only the ones >= 3**

In [39]:
n_viaggi = dog_travel.groupby('id').count()
n_viaggi.rename(columns={'index' : 'n_travels'}, inplace=True)
n_viaggi = n_viaggi[['n_travels']]
n_viaggi.head()

,n_travels
id,
8619716,2
9317153,1
12134429,1
14355301,1
16313278,1


In [40]:
n_viaggi = n_viaggi[n_viaggi['n_travels'] >= 3]
n_viaggi.head()

,n_travels
id,
16657005,4
20905974,5
24894870,4
24894894,4
33218331,7


**Merging with the 'dogs_clean' df, we can find the 'breed_primary'**

In [41]:
atleast_3 = pd.merge(n_viaggi, dogs_clean, on='id')
atleast_3[['id', 'n_travels', 'breed_primary']].head()

,id,n_travels,breed_primary
0,16657005,4,Pit Bull Terrier
1,20905974,5,Chow Chow
2,24894870,4,Hound
3,24894894,4,Hound
4,33218331,7,Alaskan Malamute


In [42]:
len(atleast_3)

563

# 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [43]:
dog_travel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [44]:
"""
The function takes two strings in input 'found' and 'manual'
and returns 'found' if 'manual' is Null
returns 'manual' otherwise
"""
def correct_state(found, manual):
    if pd.isna(manual):
        return found
    else:
        return manual

**Apply the above function to every row of the 'dog_travel' dataframe using the lambda row sintax and storing the result in a list called 'lista'**

In [45]:
lista = dog_travel.apply(lambda row: correct_state(row['found'], row['manual']), axis=1)
lista

0          Arkansas
1           Bahamas
2          Maryland
3           Adaptil
4       Afghanistan
           ...     
6189             WV
6190        Wyoming
6191         Yazmin
6192           Ohio
6193           Zazu
Length: 6194, dtype: object

**Replacing the values in 'lista'**

In [46]:
dog_travel['found'] = lista
dog_travel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


# 9. For each state, compute the ratio between the number of travels and the population.

In [47]:
dog_travel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


**We merge the 'NST_EST' df and the 'states' df to have a complete df with 'abbreviation', 'state', 'population'**

In [48]:
NST = pd.merge(NST_EST, states, on='state')
NST[['abbreviation', 'state', 'population']].head()

,abbreviation,state,population
0,AL,Alabama,5024279
1,AK,Alaska,733391
2,AZ,Arizona,7151502
3,AR,Arkansas,3011524
4,CA,California,39538223


**We count the number of travels for each state**

In [49]:
travels_count = dog_travel.groupby('contact_state').size().reset_index(name='travels')
travels_count.head()

,contact_state,travels
0,17325,10
1,AL,75
2,AR,10
3,AZ,70
4,CA,28


**We notice that '17325' correspond to the zip code of Pennsylvania (PA), so we replace it and run the count again**

In [50]:
dog_travel['contact_state'].replace('17325', 'PA', inplace=True)

In [51]:
travels_count = dog_travel.groupby('contact_state').size().reset_index(name='travels')
travels_count

,contact_state,travels
0,AL,75
1,AR,10
2,AZ,70
3,CA,28
4,CO,103
5,CT,90
6,DC,112
7,DE,57
8,FL,133
9,GA,109


**We merge as to have a df with 'contact_state', 'population', 'travels' so now we can calculate the ratio in a new column**

In [52]:
state_ratio = pd.merge(NST, travels_count, left_on='abbreviation', right_on='contact_state')[['contact_state','population', 'travels']]
state_ratio.head()

,contact_state,population,travels
0,AL,5024279,75
1,AZ,7151502,70
2,AR,3011524,10
3,CA,39538223,28
4,CO,5773714,103


In [53]:
state_ratio['ratio'] = state_ratio['travels'] / state_ratio['population']
state_ratio.rename(columns = {'contact_state' : 'state'}, inplace = True)
state_ratio.head()

,state,population,travels,ratio
0,AL,5024279,75,1.492751e-05
1,AZ,7151502,70,9.788154e-06
2,AR,3011524,10,3.320578e-06
3,CA,39538223,28,7.081755e-07
4,CO,5773714,103,1.783947e-05


# 10. For each dog, compute the number of days from the posted day to the day of last access.


**Control data types of 'posted' and 'accessed' columns and changing them to 'datetime' if they are not**

In [54]:
print(dogs_clean['posted'].dtypes)

datetime64[ns, UTC]


In [55]:
print(dogs_clean['accessed'].dtypes)

object


In [56]:
dogs_clean['accessed'] = pd.to_datetime(dogs_clean['accessed'], errors = 'coerce')

In [57]:
dogs.dropna(subset=['accessed'], inplace=True)
len(dogs_clean)

58180

In [58]:
print(dogs_clean['posted'].dtypes)

datetime64[ns, UTC]


In [59]:
print(dogs_clean['accessed'].dtypes)

datetime64[ns]


**Converting from a 'datetime' to a 'date'**

In [60]:
dogs_clean['posted'] = dogs_clean['posted'].dt.date

In [61]:
dogs_clean['accessed'] = dogs_clean['accessed'].dt.date

**Calculating the days between 'posted' and 'accessed'**

In [62]:
dogs_clean['days_between'] = (dogs_clean['accessed'] - dogs_clean['posted']).dt.days

In [63]:
days_between = dogs_clean[['id', 'posted', 'accessed', 'days_between']]
days_between

,id,posted,accessed,days_between
0,46042150,2019-09-20,2019-09-20,0
1,46042002,2019-09-20,2019-09-20,0
2,46040898,2019-09-20,2019-09-20,0
3,46039877,2019-09-20,2019-09-20,0
4,46039306,2019-09-20,2019-09-20,0
...,...,...,...,...
58175,44605893,2019-05-03,2019-09-20,140
58176,44457061,2019-04-13,2019-09-20,160
58177,42865848,2018-09-27,2019-09-20,358
58178,42734734,2018-09-12,2019-09-20,373


# 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

**We devide the days that have passed by 7, as to have the number of weeks**

In [64]:
dogs_clean['weeks_between'] = (dogs_clean['accessed'] - dogs_clean['posted']).dt.days/7
dogs_clean['weeks_between'] = dogs_clean['weeks_between'].round(2)

**We then bin the number of weeks into pre-defined ranges, and save to a new column**

In [65]:
dogs_clean['weeks_range'] = pd.cut(dogs_clean['weeks_between'], bins=[-np.inf, 1, 4, 8, 12, 16, 24, 25, 52, np.inf], labels=['< 1 week', '1-4 weeks', '4-8 weeks', '8-12 weeks', '12-16 weeks', '16 weeks - 6 months', '> 6 months', '1 year', '> 1 year'])

In [66]:
weeks_between = dogs_clean[['id', 'posted', 'accessed', 'days_between', 'weeks_between', 'weeks_range']]
weeks_between

,id,posted,accessed,days_between,weeks_between,weeks_range
0,46042150,2019-09-20,2019-09-20,0,0.00,< 1 week
1,46042002,2019-09-20,2019-09-20,0,0.00,< 1 week
2,46040898,2019-09-20,2019-09-20,0,0.00,< 1 week
3,46039877,2019-09-20,2019-09-20,0,0.00,< 1 week
4,46039306,2019-09-20,2019-09-20,0,0.00,< 1 week
...,...,...,...,...,...,...
58175,44605893,2019-05-03,2019-09-20,140,20.00,16 weeks - 6 months
58176,44457061,2019-04-13,2019-09-20,160,22.86,16 weeks - 6 months
58177,42865848,2018-09-27,2019-09-20,358,51.14,1 year
58178,42734734,2018-09-12,2019-09-20,373,53.29,> 1 year


# 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [67]:
# Drop any rows with missing values
dogs_sub = dogs_clean.dropna(subset=['breed_primary', 'description', 'sex'])
dogs_sub

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,days_between,weeks_between,weeks_range
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,Yellow / Tan / Blond / Fawn,NaN,Senior,Male,Medium,Short,True,True,NaN,False,True,NaN,NaN,NaN,HARLEY,adoptable,2019-09-20,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,0,0.00,< 1 week
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,White / Cream,NaN,Adult,Male,Large,Short,True,True,NaN,False,True,NaN,NaN,NaN,BIGGIE,adoptable,2019-09-20,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07,0,0.00,< 1 week
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,NaN,NaN,Adult,Male,Large,Short,True,False,NaN,False,True,NaN,NaN,NaN,Ziggy,adoptable,2019-09-20,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51,0,0.00,< 1 week
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,NaN,NaN,Young,Male,Small,Long,True,False,NaN,False,True,True,True,True,Theo,adoptable,2019-09-20,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09,0,0.00,< 1 week
5,46039304,NV184,https://www.petfinder.com/dog/oliver-46039304/...,Dog,Dog,Boxer,Beagle,True,False,NaN,NaN,NaN,Baby,Male,Medium,Short,True,False,NaN,False,True,True,True,True,Oliver,adoptable,2019-09-20,Henderson,NV,89052,US,89009,2019-09-20,Dog,Oliver was born around mid-June and came to us...,70,97.77,0,0.00,< 1 week
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58166,45168741,WY24,https://www.petfinder.com/dog/charlie-45168741...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,NaN,NaN,Adult,Male,Medium,NaN,True,False,NaN,False,False,True,True,True,Charlie,adoptable,2019-07-04,Riverton,WY,82501,US,WY,2019-09-20,Dog,"Charlie was adopted from us 5 years ago, but r...",109,280.51,78,11.14,8-12 weeks
58167,44843897,WY24,https://www.petfinder.com/dog/samson-44843897/...,Dog,Dog,Pit Bull Terrier,NaN,False,False,NaN,NaN,NaN,Adult,Male,Large,NaN,True,False,NaN,False,False,True,True,True,Samson,adoptable,2019-05-31,Riverton,WY,82501,US,WY,2019-09-20,Dog,Samson is a dog that will need someone to show...,74,132.32,112,16.00,12-16 weeks
58172,44658860,WY24,https://www.petfinder.com/dog/buddy-44658860/w...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,NaN,NaN,NaN,Baby,Male,Medium,NaN,True,True,NaN,False,False,True,True,True,Buddy,adoptable,2019-05-10,Riverton,WY,82501,US,WY,2019-09-20,Dog,Buddy was an owner surrender by an older gentl...,135,357.97,133,19.00,16 weeks - 6 months
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",NaN,NaN,Adult,Male,Medium,Medium,True,True,NaN,False,False,NaN,NaN,NaN,Tren,adoptable,2019-05-03,Lander,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34,140,20.00,16 weeks - 6 months


In [68]:
# Preprocess the description field
def preprocess_description(desc):
    if pd.isna(desc):
        return []
    desc = re.sub(r'\W+', ' ', desc.lower())
    return desc.split()

^ this function takes a string as input and returns a list of preprocessed words. The preprocessing steps include converting the string to lowercase, removing any non-alphanumeric characters, and splitting the string into a list of words.

In [69]:
# Define a function to calculate the percentage of shared words between two descriptions

def shared_word_percentage(desc1, desc2):
    set1 = set(desc1)
    set2 = set(desc2)
    if len(set1) == 0 or len(set2) == 0:
        return 0.0
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

^ this function is defined to calculate the percentage of shared words between two descriptions. This is done by first converting the descriptions to sets of words, and then calculating the intersection and union of the sets.


In [70]:
dogs_sub.loc[:, 'description_processed'] = dogs_sub['description'].apply(preprocess_description)

C:\Users\david\AppData\Local\Temp\ipykernel_7248\3402708723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogs_sub.loc[:, 'description_processed'] = dogs_sub['description'].apply(preprocess_description)


^ the 'description_processed' column in the dogs_clean DataFrame is created by applying the 'preprocess_description' function to the 'description' column using the 'apply()' method.

In [71]:
# Create a dictionary to store the processed descriptions and their corresponding IDs
desc_dict = dict(zip(dogs_sub['id'], dogs_sub['description_processed']))
#desc_dict

^ a dictionary called 'desc_dict' is created to store the processed descriptions and their corresponding IDs.

In [72]:
# Use a hash table to store the processed descriptions and their corresponding IDs
desc_hash = {}
for i, desc in enumerate(dogs_sub['description_processed']):
    desc_hash[hash(' '.join(desc))] = desc_hash.get(hash(' '.join(desc)), []) + [dogs_sub.iloc[i]['id']]

^ a hash table called 'desc_hash' is created to store the processed descriptions and their corresponding IDs. The hash table is constructed by iterating over the 'description_processed' column in the dogs_sub DataFrame, and using the 'hash()' function to generate a hash value for each description. The hash value is used as a key in the desc_hash dictionary, and the corresponding ID is added to a list of IDs associated with that hash value.

In [73]:
# Find the duplicate pairs using the hash table
duplicate_pairs = []
for i, desc in enumerate(dogs_sub['description_processed']):
    desc_hash[hash(' '.join(desc))].remove(dogs_sub.iloc[i]['id'])
    for j in desc_hash[hash(' '.join(desc))]:
        if dogs_sub.iloc[i]['breed_primary'] == dogs_sub[dogs_sub['id'] == j]['breed_primary'].values[0] and \
           dogs_sub.iloc[i]['breed_secondary'] == dogs_sub[dogs_sub['id'] == j]['breed_secondary'].values[0] and \
           dogs_sub.iloc[i]['sex'] == dogs_sub[dogs_sub['id'] == j]['sex'].values[0]:
            shared_words = shared_word_percentage(desc, desc_dict[j])
            if shared_words >= 0.9:
                duplicate_pairs.append((dogs_sub.iloc[i]['id'], j))
    desc_hash[hash(' '.join(desc))].append(dogs_sub.iloc[i]['id'])

^ a loop is used to iterate over the 'description_processed' column in the dogs_sub DataFrame. For each record, the ID is removed from the list of IDs associated with its hash value in the desc_hash dictionary. Then, a nested loop is used to compare the current record to all other records in the desc_hash dictionary with the same hash value. If the current record and another record have the same breeds and sex, and their shared word percentage is at least 90%, then they are considered a duplicate pair and their IDs are added to the duplicate_pairs list.

^^ execution time: ~ 1 minute

**We save the pairs in a new df and populate with the corresponding values**

In [74]:
df_duplicate_pairs = pd.DataFrame(duplicate_pairs, columns=['id_1', 'id_2'])
df_duplicate_pairs.head()

,id_1,id_2
0,45901985,45901991
1,45901985,45901992
2,45901985,45901994
3,45901985,45901995
4,45901985,45901993


In [75]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = 40

In [76]:
dogs_sub[dogs_sub['id'] == 44696946][['id', 'breed_primary', 'breed_secondary', 'sex', 'description']]

,id,breed_primary,breed_secondary,sex,description
57893,44696946,Pit Bull Terrier,Hound,Male,Color:black/white


In [77]:
dogs_sub[dogs_sub['id'] == 45301676][['id', 'breed_primary', 'breed_secondary', 'sex', 'description']]

,id,breed_primary,breed_secondary,sex,description
57779,45301676,Pit Bull Terrier,Hound,Male,Color:black/white


In [78]:
# Merge 'df_duplicate_pairs' with 'dogs' to populate it with the corresponding values
df_duplicate_pairs = pd.merge(df_duplicate_pairs, dogs, left_on='id_1', right_on='id')
df_duplicate_pairs = df_duplicate_pairs[['id_1', 'id_2', 'breed_primary', 'breed_secondary', 'sex', 'description']]
df_duplicate_pairs = pd.merge(df_duplicate_pairs, dogs, suffixes=['_1', '_2'], left_on='id_2', right_on='id')
df_duplicate_pairs = df_duplicate_pairs[['id_1', 'id_2', 'breed_primary_1', 'breed_primary_2', 'breed_secondary_1', 'breed_secondary_2', 'sex_1', 'sex_2', 'description_1', 'description_2']]

# Print the resulting DataFrame
df_duplicate_pairs.head()

,id_1,id_2,breed_primary_1,breed_primary_2,breed_secondary_1,breed_secondary_2,sex_1,sex_2,description_1,description_2
0,45901985,45901991,Collie,Collie,Shepherd,Shepherd,Male,Male,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link
1,45901992,45901991,Collie,Collie,Shepherd,Shepherd,Male,Male,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link
2,45901994,45901991,Collie,Collie,Shepherd,Shepherd,Male,Male,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link
3,45901995,45901991,Collie,Collie,Shepherd,Shepherd,Male,Male,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link
4,45901993,45901991,Collie,Collie,Shepherd,Shepherd,Male,Male,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link,On hold until 9/27 when second set of shots are given. $50 deposit can hold. Non-refundable deposit can be made by going to www.NevadaPaws.org and following the link
